In [ ]:
# !pip install tensorflow

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

pd.set_option("display.max_columns",None)

In [ ]:
import os
import pandas as pd


main_folder = "/content/drive/MyDrive/data"

all_patients_data = []

for patient_folder in os.listdir(main_folder):
    patient_path = os.path.join(main_folder, patient_folder)

    if os.path.isdir(patient_path):

        blood_file = os.path.join(patient_path, "blood_monitoring.csv")
        bp_file = os.path.join(patient_path, "bp_monitoring.csv")
        lab_file = os.path.join(patient_path, "lab_results.csv")


        blood_df = pd.read_csv(blood_file)
        bp_df = pd.read_csv(bp_file)
        lab_df = pd.read_csv(lab_file)


        merged_df = blood_df.merge(bp_df, on=["Patient_ID", "Date", "Time"], how="outer")


        lab_columns = set(lab_df.columns) - {"Patient_ID", "Date"}


        existing_columns = lab_columns.intersection(merged_df.columns)


        lab_df = lab_df.drop(columns=existing_columns, errors="ignore")


        merged_df = merged_df.merge(lab_df, on=["Patient_ID", "Date"], how="outer")


        merged_df = merged_df.sort_values(by=["Patient_ID", "Date"], ascending=False)


        merged_df[list(existing_columns)] = merged_df[list(existing_columns)].ffill()


        all_patients_data.append(merged_df)
final_df = pd.concat(all_patients_data, ignore_index=True)
final_df=final_df.drop("Lab Result",axis=1)

final_output_path = "merged_patient_data.csv"
final_df.to_csv(final_output_path, index=False)

print(f"Final merged dataset saved at: {final_output_path}")


Final merged dataset saved at: merged_patient_data.csv


In [ ]:
df=pd.read_csv("/content/merged_patient_data.csv")

In [ ]:
df.head()

,Date,Time,Blood Glucose Level (mg/dL),Blood Oxygen (SpO₂),Electrocardiogram (ECG/EKG),Hydration Levels,Patient_ID,Blood Pressure,Heart Rate (HR),Respiratory Rate (RR),Body Temperature,Hemoglobin,Glucose,Cholesterol,Heart Rate,Platelet Count,WBC Count,RBC Count,Creatinine,Urea,Sodium,Potassium,Calcium
0,28-02-2025,0.07.33,125.3,95.4,1.18,67.0,P12346,95/84,62,15,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
1,28-02-2025,0.17.33,93.9,99.3,1.15,62.9,P12346,95/83,89,13,36.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
2,28-02-2025,0.27.33,185.0,97.2,0.89,59.4,P12346,122/67,69,17,37.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
3,28-02-2025,0.37.33,108.1,98.1,0.93,70.2,P12346,140/62,76,13,37.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
4,28-02-2025,0.47.33,127.6,97.3,0.89,55.9,P12346,135/80,60,19,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11


In [ ]:
df.columns

Index(['Date', 'Time', 'Blood Glucose Level (mg/dL)', 'Blood Oxygen (SpO₂)',
       'Electrocardiogram (ECG/EKG)', 'Hydration Levels', 'Patient_ID',
       'Blood Pressure', 'Heart Rate (HR)', 'Respiratory Rate (RR)',
       'Body Temperature', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')

In [ ]:
# preprocessing data

df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H.%M.%S')
df = df.drop(columns=['Date', 'Time','Glucose','Heart Rate'])
df = df.sort_values(by='Timestamp').reset_index(drop=True)
df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)
df = df.drop(columns=['Blood Pressure'])
df = df.fillna(method='ffill')



scaler = MinMaxScaler()
def normalize_group(group):
    df_numeric = group.drop(columns=['Patient_ID', 'Timestamp'])
    group[df_numeric.columns] = scaler.fit_transform(df_numeric)
    return group

df = df.groupby('Patient_ID', group_keys=False).apply(normalize_group)


def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

predictions = {}
for patient_id, group in df.groupby('Patient_ID'):
    data_values = group.drop(columns=['Patient_ID', 'Timestamp']).values


    X, y = create_sequences(data_values, seq_length=5)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(5, X.shape[2])),
        LSTM(50, activation='relu'),
        Dense(X.shape[2])
    ])


    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))
    model.save(f"patient_{patient_id}_lstm_model.keras")
    last_sequence = X[-1].reshape(1, 5, X.shape[2])
    next_row_prediction = model.predict(last_sequence)
    next_row_original = scaler.inverse_transform(next_row_prediction)


    predictions[patient_id] = np.round(next_row_original, 2)

for patient_id, pred in predictions.items():
    print(f"Predicted Next Row for Patient {patient_id}:")
    print(np.round(pred, 2))

<ipython-input-11-e164ab72270a>:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
<ipython-input-11-e164ab72270a>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Patient_ID', group_keys=False).apply(normalize_group)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.2603 - val_loss: 0.1165
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0843 - val_loss: 0.1113
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0604 - val_loss: 0.0998
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0506 - val_loss: 0.0916
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0475 - val_loss: 0.0954
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0482 - val_loss: 0.0953
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0459 - val_loss: 0.0928
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0484 - val_loss: 0.0959
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0442 - val_loss: 0.0946
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0452 - val_loss: 0.0948
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0445 - val_loss: 0.0970
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0453 - va

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.2382 - val_loss: 0.1610
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0592 - val_loss: 0.1601
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0464 - val_loss: 0.1586
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0441 - val_loss: 0.1609
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0431 - val_loss: 0.1559
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0434 - val_loss: 0.1590
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0419 - val_loss: 0.1627
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0429 - val_loss: 0.1556
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0432 - val_loss: 0.1620
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0426 - val_loss: 0.1614
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0414 - val_loss: 0.1527
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0416 - val_loss: 0.1609

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.2678 - val_loss: 0.1440
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0735 - val_loss: 0.1271
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0530 - val_loss: 0.1242
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0479 - val_loss: 0.1241
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0463 - val_loss: 0.1256
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0448 - val_loss: 0.1285
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0446 - val_loss: 0.1292
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0448 - val_loss: 0.1267
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0441 - val_loss: 0.1275
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0436 - val_loss: 0.1260
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0428 - val_loss: 0.1240
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0437 - val_loss: 0.

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

seq_length = 5  # Same as used during training
predictions = {}

for patient_id, group in df.groupby('Patient_ID'):
    # Load the trained model for the specific patient
    model_path = f"patient_{patient_id}_lstm_model.keras"

    try:
        model = load_model(model_path)  # Load the saved model
    except Exception as e:
        print(f"Model not found for Patient {patient_id}, skipping...")
        continue

    # Get the last `seq_length` rows for this patient
    latest_data = group.drop(columns=['Patient_ID', 'Timestamp']).values

    if len(latest_data) < seq_length:
        print(f"Not enough data for Patient {patient_id}, skipping...")
        continue  # Skip if not enough data for sequence

    last_sequence = latest_data[-seq_length:].reshape(1, seq_length, latest_data.shape[1])  # Reshape for LSTM input

    # Make the prediction
    predicted_values = model.predict(last_sequence)

    # Reverse normalization to get original values
    predicted_values_original = scaler.inverse_transform(predicted_values)

    # Store the prediction
    predictions[patient_id] = np.round(predicted_values_original, 2)

# Print predictions for all patients
for patient_id, pred in predictions.items():
    print(f"Predicted Next Row for Patient {patient_id}:")
    print(pred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
Predicted Next Row for Patient P12332:
[[1.4255000e+02 9.7300003e+01 1.0200000e+00 6.0580002e+01 7.9470001e+01
  1.5620000e+01 3.6770000e+01 1.5230000e+01 2.1464000e+02 3.0578616e+05
  4.1081099e+03 5.6599998e+00 9.8000002e-01 2.9270000e+01 1.3896001e+02
  4.3099999e+00 1.0010000e+01 1.1717000e+02 7.4589996e+01]]
Predicted Next Row for Patient P12345:
[[1.2869000e+02 9.7080002e+01 1.0200000e+00 6.0779999e+01 7.7550003e+01
  1.5820000e+01 3.6919998e+01 1.6219999e+01 1.8709000e+02 2.4413981e+05
  7.1709702e+03 4.9400001e+00 8.2999998e-01 1.9920000e+01 1.4000999e+02
  3.8800001e+00 1.0090000e+01 1.1629000e+02 7.2860001e+01]]
Predicted Next Row for Patient P12346:
[[1.3656000e+02 9.8070000e+01 1.0100000e+00 6.3619999e+01 8.1239998e+01
  1.5380000e+01 3.7000000e+01 1.4720000e+01 1.6959000e+02 2.6230175e+05
  6.6421299e+03 5.1199999e+00 6.9999999e-01 3.4529999e+01 1.3884000e+02
  4.0000000e+00 9.9099998e+00 1.1670000e+02 7.6000000e+01]]


In [ ]:
df1=pd.read_csv("/content/merged_patient_data.csv")

In [ ]:
df1.head()

,Date,Time,Blood Glucose Level (mg/dL),Blood Oxygen (SpO₂),Electrocardiogram (ECG/EKG),Hydration Levels,Patient_ID,Blood Pressure,Heart Rate (HR),Respiratory Rate (RR),Body Temperature,Hemoglobin,Glucose,Cholesterol,Heart Rate,Platelet Count,WBC Count,RBC Count,Creatinine,Urea,Sodium,Potassium,Calcium
0,28-02-2025,0.07.33,151.211469,94.993958,1.010389,74.339280,P12332,108/76,63,16,36.528112,12.958354,203.353464,247.433986,64.0,295839.0,4054.199744,6.01111,1.390159,41.857931,144.61325,4.829715,8.623545
1,28-02-2025,0.17.33,158.056759,95.364928,1.000853,69.503060,P12332,136/80,91,13,37.754934,12.958354,203.353464,247.433986,64.0,295839.0,4054.199744,6.01111,1.390159,41.857931,144.61325,4.829715,8.623545
2,28-02-2025,0.27.33,173.549089,96.875988,1.070892,57.769009,P12332,107/62,73,18,37.762628,12.958354,203.353464,247.433986,64.0,295839.0,4054.199744,6.01111,1.390159,41.857931,144.61325,4.829715,8.623545
3,28-02-2025,0.37.33,125.694898,96.177574,0.979973,59.505916,P12332,105/76,80,18,36.615731,12.958354,203.353464,247.433986,64.0,295839.0,4054.199744,6.01111,1.390159,41.857931,144.61325,4.829715,8.623545
4,28-02-2025,0.47.33,191.002260,94.045522,1.192801,69.955068,P12332,120/81,97,14,36.713499,12.958354,203.353464,247.433986,64.0,295839.0,4054.199744,6.01111,1.390159,41.857931,144.61325,4.829715,8.623545


In [ ]:
df1.columns

Index(['Date', 'Time', 'Blood Glucose Level (mg/dL)', 'Blood Oxygen (SpO₂)',
       'Electrocardiogram (ECG/EKG)', 'Hydration Levels', 'Patient_ID',
       'Blood Pressure', 'Heart Rate (HR)', 'Respiratory Rate (RR)',
       'Body Temperature', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')